In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import wordnet

In [2]:
data=pd.read_csv(r"C:\Users\YUE\Desktop\ui6.csv")#encoding='ISO-8859-1'
#data[['NUM','REVIEWS']]=data[['NUM','REVIEWS']].astype(str)

In [3]:
#str化
def split_text(text):
    pure_text=''
    for letter in text:
        pure_text+=letter
    return text

In [4]:
docs=data['Reviews']
docs=docs.apply(lambda s:split_text(s))

In [5]:
doclist=docs.values

In [6]:
texts=[[word for word in doc.split()] for doc in doclist]
for sentence in texts:
    for word in sentence:
        if word=="audit":
            print("ok")

In [7]:
from gensim import corpora,models,similarities
import gensim

In [8]:
#建立语料库 把每个单词用一个数字的index来指代，把原文本变成成一条长长的数组
dictionary=corpora.Dictionary(texts)
#dictionary.filter_n_most_frequent(1)
corpus=[dictionary.doc2bow(text) for text in texts]


In [9]:
#计算文档TF_IDF
#corpus_tfidf=models.TfidfModel(corpus)[corpus]

In [10]:
LDA=gensim.models.ldamodel.LdaModel(corpus,id2word=dictionary,num_topics=3,random_state=100,
                chunksize=1000, passes=50,iterations=500)

In [11]:
LDA.print_topics(num_topics=3,num_words=7)

[(0,
  '0.012*"still" + 0.011*"issue" + 0.011*"play" + 0.011*"interface" + 0.011*"picture" + 0.010*"design" + 0.010*"get"'),
 (1,
  '0.033*"comment" + 0.023*"layout" + 0.018*"fix" + 0.016*"change" + 0.015*"section" + 0.012*"make" + 0.010*"get"'),
 (2,
  '0.021*"back" + 0.017*"like" + 0.017*"layout" + 0.013*"use" + 0.013*"watch" + 0.013*"go" + 0.011*"screen"')]

In [2]:
#import pyLDAvis.gensim_models

In [3]:
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim_models.prepare(LDA, corpus, dictionary)
#vis

In [61]:
#困惑度计算
def perplexity(ldamodel, testset, dictionary, size_dictionary, num_topics):
    prep = 0.0
    prob_doc_sum = 0.0
    topic_word_list = [] 
    for topic_id in range(num_topics):
        topic_word = ldamodel.show_topic(topic_id, size_dictionary)
        dic = {}
        for word, probability in topic_word:
            dic[word] = probability
        topic_word_list.append(dic)
    doc_topics_ist = [] 
    for doc in testset:
        doc_topics_ist.append(ldamodel.get_document_topics(doc, minimum_probability=0))
    testset_word_num = 0
    for i in range(len(testset)):
        prob_doc = 0.0 
        doc = testset[i]
        doc_word_num = 0 
        for word_id, num in dict(doc).items():
            prob_word = 0.0 
            doc_word_num += num
            word = dictionary[word_id]
            for topic_id in range(num_topics):
                prob_topic = doc_topics_ist[i][topic_id][1]
                prob_topic_word = topic_word_list[topic_id][word]
                prob_word += prob_topic*prob_topic_word
            prob_doc += math.log(prob_word) 
        prob_doc_sum += prob_doc
        testset_word_num += doc_word_num
    prep = math.exp(-prob_doc_sum/testset_word_num)
    return prep

In [18]:
import math
import matplotlib.pyplot as plt

In [19]:
#topic个数hepreplexity的关系图
topic=[]
prep_value=[]

for i in range(1,20,1):
    topic.append(str(i))
    ldamodel=gensim.models.ldamodel.LdaModel(corpus,id2word=dictionary,num_topics=i,random_state=100,
               chunksize=1000, passes=50,iterations=100)
    testset=[]
    num_topics=i
    for j in range(int(1515/15)):
        testset.append(corpus[j*15])
    dictionary=corpora.Dictionary(texts)
    size_dictionary=len(dictionary.keys())
    prep=perplexity(ldamodel, testset, dictionary,size_dictionary,num_topics)
    prep_value.append(prep)
    
plt.plot(topic,prep_value,linewidth=2,color="red")
plt.xlabel("Number of topic")
plt.ylabel("Preplexity")
plt.show()

IndexError: list index out of range